In [1]:
import pandas as pd
import numpy as np
import tarfile
import io
import glob
import dask.dataframe as dd

# import xgboost as xgb
# from xgboost import XGBClassifier
from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

import lightgbm as lgb
import warnings
warnings.simplefilter("ignore")



tar = tarfile.open('../data/raw/20181120_track_features.tar.gz', 'r:gz')
csv_files = tar.getnames()

tf_df_list = []

for csv_file in [csv_files[2], csv_files[4]]:
    csv_contents = tar.extractfile(csv_file).read()
    tf_df_list.append(pd.read_csv(io.BytesIO(csv_contents), encoding='utf8'))

tf_df = pd.concat(tf_df_list, ignore_index=True)
tf_df.rename(columns={'track_id':'track_id_clean'}, inplace=True)

kmean300_df = pd.read_csv('../data/interim/all_data/mbKMeans300clusters.csv', usecols=['track_id','clus'])
kmean300_df.rename(columns={'track_id':'track_id_clean'}, inplace=True)



In [2]:
def get_sim(df_hist, df_lookup, sim_file_list, score_name_list):
    df_hist['ListenYes'] = (df_hist['skip_2'] == False)*1
    df_hist['ListenYes'].replace(0, -1, inplace = True)
    df_hist = df_hist.groupby(['session_id', 'clus']).agg({'ListenYes':['sum']})
    df_hist = df_hist.reset_index()
    df_hist.columns = df_hist.columns.droplevel(level = 1) # take out the unwanted level
    df_pivot = pd.pivot_table(df_hist, values = 'ListenYes',index='session_id', columns='clus')
    df_pivot = df_pivot.fillna(0)
    
    
    for sim_file, score_name in zip(sim_file_list, score_name_list):
        sim_matrix = pd.read_csv(sim_file).drop(columns=['Unnamed: 0'])
        sim_matrix.columns = list(map(str, range(0,len(sim_matrix))))
        df_sim_session = df_pivot.dot(sim_matrix)/sim_matrix.sum()
        
        df_lookup[score_name] = df_sim_session.lookup(df_lookup['session_id'],df_lookup['clus'].astype(str))
    
    return df_lookup

In [3]:

file_list = glob.glob('../data/raw/training_set/log_9*.csv')
    

In [4]:
col_FA = ['skip_1', 'skip_2', 'skip_3', 'not_skipped', 'context_switch','hour_of_day','premium',
       'no_pause_before_play', 'short_pause_before_play',
       'long_pause_before_play', 'hist_user_behavior_n_seekfwd',
       'hist_user_behavior_n_seekback', 'hist_user_behavior_is_shuffle',
       'hist_user_behavior_reason_end_backbtn',
       'hist_user_behavior_reason_end_clickrow',
       'hist_user_behavior_reason_end_endplay',
       'hist_user_behavior_reason_end_fwdbtn',
       'hist_user_behavior_reason_end_logout',
       'hist_user_behavior_reason_end_remote',
       'hist_user_behavior_reason_end_trackdone',
       'hist_user_behavior_reason_start_appload',
       'hist_user_behavior_reason_start_backbtn',
       'hist_user_behavior_reason_start_clickrow',
       'hist_user_behavior_reason_start_endplay',
       'hist_user_behavior_reason_start_fwdbtn',
       'hist_user_behavior_reason_start_playbtn',
       'hist_user_behavior_reason_start_remote',
       'hist_user_behavior_reason_start_trackdone',
       'hist_user_behavior_reason_start_trackerror', 'context_type_catalog',
       'context_type_charts', 'context_type_editorial_playlist',
       'context_type_personalized_playlist', 'context_type_radio',
       'context_type_user_collection']


In [15]:
from timeit import default_timer as timer #to see how long the computation will take


for file in file_list[0:1]:
    start = timer()
    log_df = pd.read_csv(file)
    log_df = log_df.merge(kmean300_df)

    log_df_1 = log_df.loc[log_df['session_position']<(log_df['session_length']/2)]
    log_df_1['hour_of_day'] = log_df_1['hour_of_day'].astype('float')
    log_df_1['premium'] = log_df_1['premium'].astype('bool')
    log_df_1['weekday'] = log_df_1['date'].astype('datetime64[ns]').dt.dayofweek
    log_df_1 = log_df_1.drop(columns = ['date'])
    log_df_1 = pd.get_dummies(log_df_1, columns=['hist_user_behavior_reason_end', 'hist_user_behavior_reason_start', 'context_type'])
    log_df_1_summary = log_df_1.groupby(['session_id'])[col_FA].agg(['mean'])
    log_df_1_summary.columns = log_df_1_summary.columns.get_level_values(0)+'_'+log_df_1_summary.columns.get_level_values(1)
    log_df_history = log_df_1[['session_id','track_id_clean','skip_2','clus']]


    half_cut = log_df['session_length']/2
    log_df_2 = log_df.loc[log_df['session_position']>=half_cut]
    log_df_2 = log_df_2[['session_id','track_id_clean','skip_2','session_position','session_length','clus']]
    log_df_2['weight'] = 1/(log_df['session_position']-half_cut)
    log_df_2 = log_df_2.merge(log_df_1_summary, on='session_id')


    sim_file_list = ['../models/SVD/similarity/k300_CanbDist.csv',
                     '../models/SVD/similarity/k300_CosSim.csv',
                     '../models/SVD/similarity/k300_LinCorr.csv',
                     '../models/SVD/similarity/k300_ManhDist.csv',
                     '../models/SVD/similarity/k300_HammDist.csv',
                     '../models/SVD/similarity/k300_SpearCorr.csv',
                     '../models/SVD/similarity/k300_KendCorr.csv']
    score_name_list = ['CanbDist300', 'CosSim300','LinCorr300','ManhDist300','HammDist300','SpearCorr300','KendCorr300']

    df_lookup = get_sim(log_df_history, log_df_2, sim_file_list, score_name_list)
    df_lookup = df_lookup.merge(tf_df)
    df_lookup.drop(columns = ['key','time_signature','mode'], inplace = True)
    X = df_lookup.drop(columns = ['session_id','track_id_clean','skip_2']).to_numpy()
    
    bst = lgb.Booster(model_file='../models/SVD/LightGBM_BayesOpt/LightGBM_incremental_training/boost290.txt')  # init model

    ypred = bst.predict(X)
    print('Runtime per batch: %0.2fs' % (timer() - start))





Runtime per batch: 1137.92s


In [16]:
ypred

array([1.18730635, 0.58211297, 0.58211297, ..., 2.07817067, 1.64180308,
       1.07818594])

In [13]:
from sklearn.metrics import accuracy_score
accuracy_score(ypred, df_lookup['skip_2']*1)

ValueError: Classification metrics can't handle a mix of continuous and binary targets